In [68]:
import numpy as np
import pandas as pd
import data_als
import data_prepro

In [69]:
import torch
import torch.nn as nn
from torch.autograd import Variable 

import torch.optim as optim

In [70]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [71]:
train = data_prepro.outlierDataToNan(train,low=True,high=False,thres=10.0)
# train = data_prepro.fillZero(train)
train = data_prepro.fillnaBehind(train)
train = data_prepro.fillprevValue(train)


Int64Index([17558, 21595, 25626], dtype='int64')
index                  0
datetime               0
구미 혁신도시배수지 유출유량 적산차    2
year                   0
month                  0
dtype: int64


In [72]:
x_train,y_train = data_prepro.XDataToXAndYSeq(train)

In [73]:
x_test,y_test = data_prepro.XDataToXAndYSeq(test)

In [74]:
x_train.shape, y_train.shape

((17519, 1, 24), (17519, 1))

In [99]:
x_train = x_train.reshape(-1,1,24)
y_train = y_train.reshape(-1,1)

In [100]:
y_train.shape

(17519, 1)

In [101]:
x_test = x_test.reshape(-1,1,24)
y_test = y_test.reshape(-1,1)

In [102]:
X_train_tensors = Variable(torch.Tensor(x_train))
X_test_tensors = Variable(torch.Tensor(x_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_final = torch.reshape(X_train_tensors,   X_train_tensors.shape)
X_test_tensors_final = torch.reshape(X_test_tensors,  X_test_tensors.shape) 

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([17519, 1, 24]) torch.Size([17519, 1])
Testing Shape torch.Size([8399, 1, 24]) torch.Size([8399, 1])


In [79]:
try:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device
except:
    device = "cpu"
print(torch.cuda.get_device_name(0))

Tesla M40 24GB


In [107]:
class LSTM1(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM1, self).__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.seq_length = seq_length #sequence length
 
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                      num_layers=num_layers, batch_first=True) #lstm
    self.fc_1 =  nn.Linear(hidden_size, 1024) #fully connected 1
    self.fc = nn.Linear(1024, num_classes) #fully connected last layer

    self.relu = nn.ReLU() 

  def forward(self,x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state   
    # Propagate input through LSTM

    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
   
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = self.relu(hn)
    out = self.fc_1(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc(out) #Final Output
   
    return out 

In [80]:
# class LSTM1(nn.Module):
#   def __init__(self, num_classes, input_size, hidden_size, num_layers:list, seq_length):
#     super(LSTM1, self).__init__()
#     self.num_classes = num_classes #number of classes
#     self.num_layers = num_layers #number of layers
#     self.input_size = input_size #input size
#     self.hidden_size = hidden_size #hidden state
#     self.seq_length = seq_length #sequence length
 
#     self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size[0],
#                       num_layers=num_layers, batch_first=True,bidirectional =True) #lstm
#     self.lstm2 = nn.LSTM(input_size=input_size, hidden_size=hidden_size[1],
#                       num_layers=num_layers, batch_first=True,bidirectional =True) #lstm
#     self.fc_1 =  nn.Linear(hidden_size[1]*2, 3072) #fully connected 1
#     self.fc = nn.Linear(3072, num_classes) #fully connected last layer

    
#     self.relu = nn.ReLU() 
#     self.tanh = nn.Tanh()

#   def forward(self,x):
#     h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
#     c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state   
#     # Propagate input through LSTM

#     output, (hn, cn) = self.lstm1(x, (h_0, c_0)) #lstm with input, hidden, and internal state

#     output, (hn, cn) = self.lstm2(output, (hn, cn)) #lstm with input, hidden, and internal state
#     hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
#     out = self.relu(hn)
#     out = self.fc_1(out) #first Dense
#     out = self.relu(out) #relu
#     out = self.fc(out) #Final Output
   
#     return out 

In [96]:
num_epochs = 30000 #1000 epochs
learning_rate = 0.005 #0.001 lr

input_size = 24 #number of features
hidden_size = 512 #number of features in hidden state

num_classes = 1 #number of output classes 

In [82]:
# lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to(device)

# loss_function = torch.nn.L1Loss()    # mean-squared error for regression
# optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer

In [83]:
# for epoch in range(num_epochs):
#     outputs = lstm1.forward(X_train_tensors_final.to(device)) #forward pass
#     optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    
#     # obtain the loss function
#     loss = loss_function(outputs, y_train_tensors.to(device))

#     loss.backward() #calculates the loss of the loss function
    
#     optimizer.step() #improve from loss, i.e backprop
#     if epoch%20 == 0:
#         print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()),end='')
#         outputs = lstm1.forward(X_test_tensors_final.to(device)) #forward pass
#         loss = loss_function(outputs, y_test_tensors.to(device))
#         loss.backward() #calculates the loss of the loss function
#         print("test data loss",loss.item())

In [90]:
# class simpleLSTM(nn.Module):
#   def __init__(self, num_classes, input_size, hidden_size):
#     super(simpleLSTM, self).__init__()
    
#     self.model = nn.Sequential(
#     nn.GRU(input_size,hidden_size[0],num_layers=2,batch_first = True,bidirectional=True),
#     # nn.Tanh(),/
#     # nn.Linear(hidden_size[0]*2,hidden_size[1]),
#     nn.ReLU(),
#     nn.GRU(hidden_size[1],hidden_size[1],num_layers=1,batch_first = True,bidirectional=True),
#     nn.ReLU(),
#     nn.Linear(hidden_size[1]*2,3072),
#     nn.ReLU(),
#     )
#     self.out = nn.Linear(3072,num_classes)

#   def forward(self,x):
#     y =  self.model(x)
    


In [104]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__() # 상속한 nn.Module에서 RNN에 해당하는 init 실행
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True,bidirectional =True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x): 
        # input x : (BATCH, LENGTH, INPUT_SIZE) 입니다 (다양한 length를 다룰 수 있습니다.).
        # 최초의 hidden state와 cell state를 초기화시켜주어야 합니다.
        # 배치 사이즈는 가변적이므로 클래스 내에선 표현하지 않습니다.
        # 만약 Bi-directional LSTM이라면 아래의 hidden and cell states의 첫번째 차원은 2*self.num_layers 입니다. 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # (BATCH SIZE, SEQ_LENGTH, HIDDEN_SIZE)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # hidden state와 동일

        # LSTM 순전파
        out, _ = self.lstm(x, (h0, c0)) # output : (BATCH_SIZE, SEQ_LENGTH, HIDDEN_SIZE) tensors. (hn, cn)은 필요 없으므로 받지 않고 _로 처리합니다. 

        # 마지막 time step(sequence length)의 hidden state를 사용해 Class들의 logit을 반환합니다(hidden_size -> num_classes). 
        out = self.fc(out[:, -1, :])
        return out

In [108]:
simpleLSTM = LSTM1(input_size, hidden_size,3,num_classes).to(device)

train_loss_function = torch.nn.MSELoss()    # mean-squared error for regression
test_loss_function = torch.nn.L1Loss()
optimizer = torch.optim.Adam(simpleLSTM.parameters(), lr=learning_rate)  # adam optimizer

TypeError: LSTM1.__init__() missing 1 required positional argument: 'seq_length'

In [109]:
for epoch in range(num_epochs):
    outputs = simpleLSTM.forward(X_train_tensors_final.to(device)) #forward pass
    loss = train_loss_function(outputs, y_train_tensors.to(device))

    if epoch%20 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()),end=', ')
        outputs = simpleLSTM.forward(X_test_tensors_final.to(device)) #forward pass
        test_loss = test_loss_function(outputs, y_test_tensors.to(device))
        # loss.backward() #calculates the loss of the loss function

        print("test data loss",test_loss.item())

    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    loss.backward() #calculates the loss of the loss function
    optimizer.step()

RuntimeError: Expected hidden[0] size (6, 17519, 512), got [3, 17519, 512]